The parameter engine='openpyxl' is specified in pd.read_excel() to explicitly tell pandas to use the OpenPyXL library for reading .xlsx files

In [21]:
import pandas as pd

# Load dataset
df = pd.read_excel('../data/Online_Retail.xlsx', engine='openpyxl')

# Show basic info
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [22]:
# Check null counts
df.isnull().sum()



InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Since description is not that necesaary so we will just remove rows with null customerID.

In [23]:
df = df.dropna(subset=['CustomerID'])


 Remove Canceled Orders
Invoices that start with 'C' are canceled

In [24]:
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

 Remove negative or zero quantity and price

In [25]:
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

Save Cleaned Data
- Setting index=False when using df.to_csv() tells pandas not to include the DataFrame's row index in the CSV file so that when sharing or importing the CSV, the index won't appear as an extra, unintended column

In [26]:
print(df.shape)
print(df.describe())
print(df['Country'].value_counts().head())

(397884, 8)
            Quantity                    InvoiceDate      UnitPrice  \
count  397884.000000                         397884  397884.000000   
mean       12.988238  2011-07-10 23:41:23.511023360       3.116488   
min         1.000000            2010-12-01 08:26:00       0.001000   
25%         2.000000            2011-04-07 11:12:00       1.250000   
50%         6.000000            2011-07-31 14:39:00       1.950000   
75%        12.000000            2011-10-20 14:33:00       3.750000   
max     80995.000000            2011-12-09 12:50:00    8142.750000   
std       179.331775                            NaN      22.097877   

          CustomerID  
count  397884.000000  
mean    15294.423453  
min     12346.000000  
25%     13969.000000  
50%     15159.000000  
75%     16795.000000  
max     18287.000000  
std      1713.141560  
Country
United Kingdom    354321
Germany             9040
France              8341
EIRE                7236
Spain               2484
Name: count, dtyp

In [27]:
import numpy as np
#0ptional: log transform can help spot outliers better
df = df[
    (df['Quantity'] < df['Quantity'].quantile(0.99)) &
    (df['UnitPrice'] < df['UnitPrice'].quantile(0.99)) &
    (df['Quantity'] > 0) &
    (df['UnitPrice'] > 0.1)
]
print(df.describe())

            Quantity                    InvoiceDate      UnitPrice  \
count  388336.000000                         388336  388336.000000   
mean        9.728171  2011-07-11 03:41:20.781436672       2.696660   
min         1.000000            2010-12-01 08:26:00       0.120000   
25%         2.000000            2011-04-07 11:30:00       1.250000   
50%         6.000000            2011-07-31 15:24:00       1.950000   
75%        12.000000            2011-10-20 15:57:00       3.750000   
max       117.000000            2011-12-09 12:50:00      14.550000   
std        13.863227                            NaN       2.462573   

          CustomerID  
count  388336.000000  
mean    15304.335877  
min     12347.000000  
25%     13975.000000  
50%     15179.000000  
75%     16805.000000  
max     18287.000000  
std      1709.012254  


In [28]:
df.to_csv('../data/Online_Retail_Clean.csv', index=False)